In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Softmax, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [2]:
print(f"{tf.__version__=}")
print(f"{np.__version__=}")
print("nvidia-smi")
!nvidia-smi
print("nvcc version")
!nvcc --version
print("nvinfer version")
!dpkg -l | grep nvinfer
print("TensorRT version")
!dpkg -l | grep TensorRT


tf.__version__='2.11.0'
np.__version__='1.24.2'
nvidia-smi
/bin/bash: nvidia-smi: command not found
nvcc version
/bin/bash: nvcc: command not found
nvinfer version
TensorRT version


# Load & Prepare Data

In [3]:
GESTRUE=["STATIC", "SLIDE_UP", "SLIDE_DOWN", "SLIDE_LEFT", "SLIDE_RIGHT", "ZOOM_IN", "ZOOM_OUT", "HIGHLIGHT", "ON_YES", "OFF_NO", "END",]

In [4]:
directory = "/content/drive/MyDrive/Colab Notebooks/rosbag/labeled_data/labeled_data_30hz"
directory = "/home/ubuntu/FYP-ROS/rosbag/data"

# get x
X = []
for filename in os.listdir(f"{directory}/data"):
    f = os.path.join(f"{directory}/data", filename)
    if os.path.isfile(f):
        x_raw = pd.read_csv(f)
        x_raw["timestamp"] -= x_raw["timestamp"][0]
        x_raw = x_raw.to_numpy()
        x_raw = np.pad(x_raw, ((0, 500 - x_raw.shape[0]), (0, 0)), 'constant')
        X.append(x_raw)

X = np.array(X)

# get y
y = []
for filename in os.listdir(f"{directory}/label"):
    f = os.path.join(f"{directory}/label", filename)
    if os.path.isfile(f):
        y.append(pd.read_csv(f)['label'])

y = tf.keras.utils.to_categorical(y, num_classes=7)

# print dimension
print(f"{X.shape=}")
print(f"{y.shape=}")

print(f"{X[0].shape=}")
print(f"{X[1].shape=}")

print(f"{X[1]=}")
print(f"{y[100]=}")

X.shape=(270, 500, 45)
y.shape=(270, 7)
X[0].shape=(500, 45)
X[1].shape=(500, 45)
X[1]=array([[0.        , 0.94      , 1.87      , ..., 0.        , 0.        ,
        1.        ],
       [0.02677846, 0.91      , 1.84      , ..., 0.        , 0.        ,
        1.        ],
       [0.05713868, 0.94      , 1.82      , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])
y[100]=array([0., 0., 0., 0., 0., 1., 0.], dtype=float32)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Build Model

In [6]:
model=Sequential()

model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(500, 45), name='BiLSTM1'))
model.add(Dropout(0.2, name='Dropout1'))
model.add(Bidirectional(LSTM(units=64, return_sequences=True, name='BiLSTM2')))
model.add(Dropout(0.2, name='Dropout2'))
model.add(Bidirectional(LSTM(units=64), name='BiLSTM3'))
model.add(Dense(7, activation='softmax', name='Dense1'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 BiLSTM1 (Bidirectional)     (None, 500, 128)          56320     
                                                                 
 Dropout1 (Dropout)          (None, 500, 128)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 500, 128)         98816     
 l)                                                              
                                                                 
 Dropout2 (Dropout)          (None, 500, 128)          0         
                                                                 
 BiLSTM3 (Bidirectional)     (None, 128)               98816     
                                                                 
 Dense1 (Dense)              (None, 7)                 903       
                                                        

In [7]:
model.load_weights("/home/ubuntu/FYP-ROS/model_lstm_weights.h5")

In [9]:
from tensorflow import keras
model2 = keras.models.load_model('/home/ubuntu/FYP-ROS/model_lstm')


# Training

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate


In [10]:
scores = model2.evaluate(X_test, y_test, verbose=0)
print(f"{scores=}")

scores=[6.782397747039795, 0.07407407462596893]


In [ ]:
y_pred = model.predict(X_test)

print(f"{np.argmax(y_pred, axis=1)=}")
print(f"{np.argmax(y_test, axis=1)=}")


In [ ]:
y_test_not_onehot = np.argmax(y_test, axis=1)
y_pred_not_onehot = np.argmax(y_pred, axis=1)
matrix_confusion = confusion_matrix(y_pred_not_onehot, y_test_not_onehot)
sns.heatmap(matrix_confusion, square=True, annot=True, cmap='Blues', fmt='d')
plt.xlabel('predictions')
plt.ylabel('ground truth')

In [ ]:
raw_data = X_test[10]
label = np.argmax(y_test[10])

fig, axs = plt.subplots(3, 6, figsize=(20, 10))

time = np.array(raw_data[:, 0])
acc_axes = axs[:, :3].ravel()
vel_axes = axs[:, 3:].ravel()
acc_data = np.concatenate([raw_data[:,1:4], raw_data[:,11:14], raw_data[:,21:24]], axis=1)
vel_data = np.concatenate([raw_data[:,4:7], raw_data[:,14:17], raw_data[:,24:27]], axis=1)
acc_titles = [f'Imu{i}_acc_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]
vel_titles = [f'Imu{i}_vel_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]

for ax, data, title in zip(acc_axes, acc_data.T, acc_titles):
    
    ax.plot(time[0:100], data[0:100])
    ax.set_title(title)
    ax.set_ylim([-1.5 * 9.8, 1.5 * 9.8])
    
for ax, data, title in zip(vel_axes, vel_data.T, vel_titles):
    ax.plot(data.flatten())
    ax.set_title(title)
    ax.set_ylim([-5, 5])

plt.plot()
fig.suptitle(f'{label}')

# Save model

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/model_lstm_weights.h5')

# Explaination 

In [ ]:
!pip install shap
import shap

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X_train.transpose(0, 2, 1))
shap.summary_plot(shap_values, X_train.transpose(0, 2, 1))